In [8]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import numpy as np
import pandas as pd
import ast
import datetime

### Functions definitions

In [10]:
# Function to facilitate filtering (removes unrealted rows, i.e, perfume, hair products, ...)
def dermSelect(df, col, keywords):
  """
  Function to facilitate filtering (removes unrealted rows, i.e, perfume, hair products, ...)
  df: dataframe to perform cleaning one
  col: column to look into for keywords
  keywords: keywords to identify rows to be deleted

  Example: df = products_data, col = "product name" , keywords = ["Perfume", "hair", "Makeup"]

  Output: dataframe with rows unrelated to problem at hand removed.
  """

  mask = df[col].apply(lambda x: not any(keyword.lower() in str(x).lower() for keyword in keywords))
  df = df[mask]
  return df

In [17]:
def extract_benefits(highlights_str):
    try:
        # Convert string representation to actual list
        highlights = ast.literal_eval(highlights_str)

        if isinstance(highlights, list):
            # Extract "good for:" benefits
            product_benefits = [item.lower().replace('good for:', '').strip()
                               for item in highlights
                               if isinstance(item, str) and item.lower().startswith('good for:')]

            # Extract "best for" skin types
            skin_types = [item.lower().replace('best for ', '').strip()
                         for item in highlights
                         if isinstance(item, str) and item.lower().startswith('best for')]

            return (', '.join(product_benefits) if product_benefits else None,
                    ', '.join(skin_types) if skin_types else None)
    except:
        return None, None
    return None, None

In [12]:
def categorize_product(row):
    """
    Three-tier categorization: secondary_category -> tertiary_category -> product_name
    """
    secondary = str(row['secondary_category']).lower() if pd.notna(row['secondary_category']) else ''
    tertiary = str(row['tertiary_category']).lower() if pd.notna(row['tertiary_category']) else ''
    name = str(row['product_name']).lower() if pd.notna(row['product_name']) else ''

    # Tier 1: Check secondary_category
    if 'eye care' in secondary:
        return 'Eye Care'
    elif 'lip balm' in secondary or 'lip treatment' in secondary:
        return 'Lip Care'
    elif 'mask' in secondary:
        return 'Mask'
    elif 'moisturizer' in secondary:
        return 'Moisturizer'
    elif 'cleanser' in secondary:
        return 'Cleanser'
    elif 'sunscreen' in secondary:
        return 'Sunscreen'
    elif 'treatment' in secondary and 'eye' not in secondary:
        # Check if it's serum-like in tertiary
        if 'serum' in tertiary:
            return 'Serum'

    # Tier 2: Check tertiary_category
    if 'eye' in tertiary and 'mask' not in tertiary:
        return 'Eye Care'
    elif 'lip' in tertiary:
        return 'Lip Care'
    elif 'mask' in tertiary:
        return 'Mask'
    elif 'peel' in tertiary:
        return 'Mask'  # Peels are similar to masks in use case
    elif 'moisturizer' in tertiary or 'cream' in tertiary:
        if 'neck' in tertiary or 'decollete' in tertiary:
            return 'Other'  # Neck creams are specialized
        return 'Moisturizer'
    elif 'serum' in tertiary or 'oil' in tertiary:
        if 'face oil' in tertiary:
            return 'Serum'  # Face oils often function like serums
        return 'Serum'
    elif 'cleanser' in tertiary or 'face wash' in tertiary or 'wipe' in tertiary:
        return 'Cleanser'
    elif 'toner' in tertiary or 'mist' in tertiary or 'essence' in tertiary:
        return 'Toner'
    elif 'sunscreen' in tertiary or 'spf' in tertiary or ' uv ' in tertiary:
        return 'Sunscreen'
    elif 'exfoliator' in tertiary:
        return 'Other'  # Exfoliators are different from masks

    # Tier 3: Check product_name
    if 'mask' in name or 'peel' in name:  # FIXED: Changed from 'mask' or 'peel' in name
        # Check if it's an eye mask or lip mask specifically
        if 'eye' in name:
            return 'Eye Care'
        elif 'lip' in name:
            return 'Lip Care'
        else:
            return 'Mask'
    elif 'eye' in name and any(keyword in name for keyword in ['cream', 'serum', 'treatment', 'gel', 'balm']):
        return 'Eye Care'
    elif 'lip' in name and any(keyword in name for keyword in ['balm', 'treatment', 'oil', 'serum', 'care']):
        return 'Lip Care'
    elif 'moisturizer' in name or 'moisturiser' in name:
        return 'Moisturizer'
    elif 'serum' in name:
        return 'Serum'
    elif 'cleanser' in name or 'cleansing' in name or 'face wash' in name:
        return 'Cleanser'
    elif 'toner' in name:
        return 'Toner'
    elif 'sunscreen' in name or 'spf' in name or ' uv' in name:
        return 'Sunscreen'
    elif 'face cream' in name or 'night cream' in name or 'day cream' in name:
        return 'Moisturizer'
    elif 'oil' in name and 'face' in name:
        return 'Serum'

    # Default to Other for anything that doesn't match
    return 'Other'

### Data Import

In [13]:
# Navigate to your data directory
data_dir = '/content/drive/MyDrive/Colab Notebooks/DermaLLM/data/'

td_date = datetime.datetime.now().strftime("%m_%d_%Y")

In [14]:
product_df = pd.read_csv(data_dir + "product_info.csv")
# drop index column
product_df.drop(['Unnamed: 0'], inplace=True, axis=1)
print(product_df.columns)
product_df

Index(['product_id', 'product_name', 'brand_id', 'brand_name', 'loves_count',
       'rating', 'reviews', 'size', 'variation_type', 'variation_value',
       'variation_desc', 'ingredients', 'price_usd', 'value_price_usd',
       'sale_price_usd', 'limited_edition', 'new', 'online_only',
       'out_of_stock', 'sephora_exclusive', 'highlights', 'primary_category',
       'secondary_category', 'tertiary_category', 'child_count',
       'child_max_price', 'child_min_price'],
      dtype='object')


,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
0,P473671,Fragrance Discovery Set,6342,19-69,6320,3.6364,11.0,NaN,NaN,NaN,...,1,0,0,"['Unisex/ Genderless Scent', 'Warm &Spicy Scen...",Fragrance,Value & Gift Sets,Perfume Gift Sets,0,NaN,NaN
1,P473668,La Habana Eau de Parfum,6342,19-69,3827,4.1538,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,85.0,30.0
2,P473662,Rainbow Bar Eau de Parfum,6342,19-69,3253,4.2500,16.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
3,P473660,Kasbah Eau de Parfum,6342,19-69,3018,4.4762,21.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
4,P473658,Purple Haze Eau de Parfum,6342,19-69,2691,3.2308,13.0,3.4 oz/ 100 mL,Size + Concentration + Formulation,3.4 oz/ 100 mL,...,1,0,0,"['Unisex/ Genderless Scent', 'Layerable Scent'...",Fragrance,Women,Perfume,2,75.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8489,P467659,Couture Clutch Eyeshadow Palette,1070,Yves Saint Laurent,2790,4.4286,7.0,NaN,NaN,NaN,...,0,0,0,NaN,Makeup,Eye,Eye Palettes,0,NaN,NaN
8490,P500874,L'Homme Eau de Parfum,1070,Yves Saint Laurent,2319,4.6367,556.0,2 oz / 60 mL,Size + Concentration + Formulation,2 oz / 60 mL eau de parfum spray,...,0,0,0,"['Layerable Scent', 'Woody & Earthy Scent']",Fragrance,Men,Cologne,1,129.0,129.0
8491,P504428,Mon Paris Eau de Parfum Gift Set,1070,Yves Saint Laurent,1475,5.0000,2.0,NaN,NaN,NaN,...,1,1,0,NaN,Fragrance,Value & Gift Sets,Perfume Gift Sets,0,NaN,NaN
8492,P504448,Y Eau de Parfum Gift Set,1070,Yves Saint Laurent,840,NaN,NaN,NaN,NaN,NaN,...,1,0,0,NaN,Fragrance,Value & Gift Sets,Cologne Gift Sets,0,NaN,NaN


### Filter out non-derma data

Remove perfumes, make up and other products that do not relate to our current problem

In [15]:
# Define keywords to filter by
rem_list = ['Fragrance','Makeup','Hair','Tools & Brushes', 'Perfume','Parfum','Bath & Body', 'Shampoo', 'Tanner', "teeth", "Body Products", "bath", "Pillowcase"]

print(product_df.shape)

# Filter out from product name
product_df_trim = dermSelect(product_df, "product_name", rem_list)
print(product_df_trim.shape)

# Filter out primary category for product use
product_df_trim = dermSelect(product_df_trim, "primary_category", rem_list)
print(product_df_trim.shape)

# Filter out secondary category for product use
product_df_trim = dermSelect(product_df_trim, "secondary_category", rem_list)
print(product_df_trim.shape)

# Filter out tertiary category for product use
product_df_trim = dermSelect(product_df_trim, "tertiary_category", rem_list)
print(product_df_trim.shape)

product_df_trim

(8494, 27)
(6813, 27)
(2652, 27)
(2432, 27)
(2416, 27)


,product_id,product_name,brand_id,brand_name,loves_count,rating,reviews,size,variation_type,variation_value,...,online_only,out_of_stock,sephora_exclusive,highlights,primary_category,secondary_category,tertiary_category,child_count,child_max_price,child_min_price
89,P439055,GENIUS Sleeping Collagen Moisturizer,6018,Algenist,33910,4.5413,1321.0,2 oz/ 60 mL,Size,2 oz/ 60 mL,...,0,0,1,"['Vegan', 'Good for: Loss of firmness', 'Colla...",Skincare,Moisturizers,Moisturizers,0,NaN,NaN
90,P421277,GENIUS Liquid Collagen Serum,6018,Algenist,67870,4.0259,1159.0,1 oz / 30 mL,Size,1 oz / 30 mL,...,0,0,1,"['Vegan', 'Good for: Loss of firmness', 'Colla...",Skincare,Treatments,Face Serums,1,25.0,25.0
91,P467602,Triple Algae Eye Renewal Balm Eye Cream,6018,Algenist,17890,4.5306,1142.0,NaN,NaN,NaN,...,0,0,1,NaN,Skincare,Eye Care,Eye Creams & Treatments,0,NaN,NaN
92,P432045,GENIUS Liquid Collagen Lip Treatment,6018,Algenist,44448,3.8721,649.0,.5 oz / 15 mL,Size,.5 oz / 15 mL,...,0,0,1,"['Vegan', 'Good for: Loss of firmness', 'Plump...",Skincare,Lip Balms & Treatments,NaN,0,NaN,NaN
93,P311143,SUBLIME DEFENSE Ultra Lightweight UV Defense F...,6018,Algenist,27278,4.4134,508.0,1 oz,Size,1 oz,...,0,0,0,"['Vegan', 'Hypoallergenic', 'UV Protection', '...",Skincare,Sunscreen,Face Sunscreen,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8424,P501474,The Youth Vault: 13-Piece Vegan Skincare + App...,6169,Youth To The People,8989,4.8750,8.0,NaN,NaN,NaN,...,1,0,1,"['Clean at Sephora', 'Hydrating', 'Good for: A...",Skincare,Value & Gift Sets,NaN,0,NaN,NaN
8425,P502998,"Youth Stacks: Plump It Up for Dry, Dehydrated ...",6169,Youth To The People,10406,4.1667,6.0,NaN,NaN,NaN,...,0,0,0,"['Vegan', 'Hyaluronic Acid', 'Clean at Sephora...",Skincare,Value & Gift Sets,NaN,0,NaN,NaN
8426,P503872,Mini Mandelic Acid + Superfood Unity Exfoliant,6169,Youth To The People,6510,4.8000,5.0,1 oz / 30 mL,Size,1 oz / 30 mL,...,0,0,0,"['Clean + Planet Positive', 'Good for: Pores',...",Skincare,Cleansers,Toners,0,NaN,NaN
8427,P502977,Youth Stacks: Daily Skin Health Your Way for P...,6169,Youth To The People,13592,4.2500,4.0,NaN,NaN,NaN,...,0,0,0,"['Vegan', 'Good for: Pores', 'Clean + Planet P...",Skincare,Value & Gift Sets,NaN,0,NaN,NaN


### Create new features

In [18]:
# Create columns based on "highlights" feature - describes the benefits of the products and skin type they are good for

product_df_trim[['product_benefits', 'skin_type_match']] = product_df_trim['highlights'].apply(lambda x: pd.Series(extract_benefits(x)))
print("New columns hit rates: " , product_df_trim[['highlights','products_benefits', 'skin_type_match']].notnull().mean())

KeyError: "['products_benefits'] not in index"

In [ ]:
# Create product type using three-tier classification approach
product_df_trim['product_type'] = product_df_trim.apply(categorize_product, axis=1)
print(product_df_trim['product_type'].notnull().mean())

In [ ]:
product_df_trim['product_type'].value_counts()

In [ ]:
# Check distribution
print(product_df_trim['product_type'].value_counts())

# Sample check for each category
for category in ['Moisturizer', 'Cleanser', 'Toner', 'Serum', 'Sunscreen', 'Eye Care', 'Lip Care','Other']:
    print(f"\n{category} examples:")
    print(product_df_trim[product_df_trim['product_type'] == category][['product_name', 'secondary_category', 'tertiary_category']].head(3))

In [ ]:
# Create price categories

product_df_trim[['price_usd','value_price_usd', 'sale_price_usd']].describe([.9, .95, .97, .98, .99])

# Base categories on 'price_usd' as more realistic on average compared to value price or sale price

In [ ]:
product_df_trim['price_range'] = None
product_df_trim.loc[product_df_trim['price_usd']<10, 'price_range'] = "< 10"
product_df_trim.loc[(product_df_trim['price_usd']>=10)&(product_df_trim['price_usd']<20), 'price_range'] = "[10 - 20)"
product_df_trim.loc[(product_df_trim['price_usd']>=20)&(product_df_trim['price_usd']<30), 'price_range'] = "[20 - 30)"
product_df_trim.loc[(product_df_trim['price_usd']>=30)&(product_df_trim['price_usd']<40), 'price_range'] = "[30 - 40)"
product_df_trim.loc[(product_df_trim['price_usd']>=40)&(product_df_trim['price_usd']<50), 'price_range'] = "[40 - 50)"
product_df_trim.loc[(product_df_trim['price_usd']>=50)&(product_df_trim['price_usd']<75), 'price_range'] = "[50 - 75)"
product_df_trim.loc[(product_df_trim['price_usd']>=75)&(product_df_trim['price_usd']<100), 'price_range'] = "[75 - 100)"
product_df_trim.loc[(product_df_trim['price_usd']>=100), 'price_range'] = ">= 100"

print(product_df_trim['price_range'].notnull().mean(), product_df_trim['price_range'].value_counts())
product_df_trim['price_range']

### Save data

In [ ]:
product_df_trim.to_csv(data_dir + f'product_data_clean_{td_date}.csv')